<a href="https://colab.research.google.com/github/atejap05/pos_data_science_furg/blob/main/disciplinas/Machine_Learning_I/semana03/Furg_ECD_03_Machine_Learning_I_Classifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning I - Classificação
### Prof. Marcelo Malheiros

Código adaptado de Aurélien Geron (licença Apache-2.0)

---

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_:

- NumPy: suporte a vetores, matrizes e operações de Álgebra Linear
- Matplotlib: biblioteca de visualização de dados
- Pandas: pacote estatístico e de manipulação de DataFrames
- Scikit-Learn: biblioteca com algoritmos de Machine Learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

# Obter os dados


Nesta atividade iremos usar o conjunto de dados MNIST, que é um conjunto de 70.000 pequenas imagens de dígitos escritos à mão. Cada imagem é identificada com o dígito que representa. Esse conjunto é bastante conhecido e também um excelente exemplo para trabalhar com tarefas de **classificação**.

O Google Colaboratory já inclui uma versão reduzida desse conjunto de dados em seu ambiente online, dentro do diretório `sample_data`. O arquivo `mnist_train_small.csv` contém o conjunto de treinamento e o arquivo `mnist_test.csv` contém o conjunto de teste, ambos rotulados com os dígitos corretos.

In [ ]:
# cada linha representa um dígito: o rótulo está na coluna 0
mnist_train = pd.read_csv('sample_data/mnist_train_small.csv', header=None)
mnist_train.shape

In [ ]:
mnist_train.head(3)

In [ ]:
# cada linha representa um dígito: o rótulo está na coluna 0
mnist_test = pd.read_csv('sample_data/mnist_test.csv', header=None)
mnist_test.shape

In [ ]:
mnist_test.head(3)

Vamos aqui separar as 784 features (pixels) de cada instância dos rótulos (dígito reconhecido). Seguiremos a convenção de **X** para features e **y** para rótulos.

Vale conferir se a separação está correta. Aqui usamos `.iloc[]` para separar uma faixa de colunas numeradas.

In [ ]:
X_train = mnist_train.iloc[:,1:]
y_train = mnist_train.iloc[:,0]
X_test  = mnist_test.iloc[:,1:]
y_test  = mnist_test.iloc[:,0]
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

## Visualizar os dados

Cada linha corresponde a uma matriz de 28 por 28 pixels (tons de cinza) correspondendo a uma única imagem de um dígito.

In [ ]:
# função auxiliar para mostrar uma imagem
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap='binary')
    plt.axis('off')

In [ ]:
# imagem associada à instância de índice 1
some_digit = X_train.iloc[1].values
some_digit_image = some_digit.reshape(28, 28)
plot_digit(some_digit_image)
plt.show()

In [ ]:
# rótulo associado à instância de índice 1
y_train.iloc[1]

In [ ]:
# função auxiliar para mostrar várias imagens
def plot_digits(instances, images_per_row):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap='binary')
    plt.axis('off')

In [ ]:
# exibição das 30 primeiras imagens
plt.figure(figsize=(9,9))
example_images = X_train.iloc[:30].values
plot_digits(example_images, images_per_row=10)
plt.show()

## Examinar a distribuição estatística dos rótulos

Vale a pena conferir se os dígitos dos dados de treino tem uma distribuição estatisticamente semelhante à base de testes.

In [ ]:
y_train.describe()

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
#y_train.hist()
y_train.hist(bins=10, range=(0,10), align='left', rwidth=0.8)
plt.show()

In [ ]:
y_test.hist(bins=10, range=(0,10), align='left', rwidth=0.8)
plt.show()

In [ ]:
# histograma combinado

#y_train.hist(bins=10, range=(0,10), align='left', rwidth=0.8)
#y_test.hist( bins=10, range=(0,10), align='left', rwidth=0.8)

y_train.hist(bins=10, range=(0,10), align='left', density=True, histtype='step')
y_test.hist( bins=10, range=(0,10), align='left', density=True, histtype='step')

plt.show()

## Criar um conjunto de testes

Esta parte já está pronta. Então o conjunto de testes será deixado de lado.

# Selecionar e treinar um modelo

Podemos ir diretamente para a parte de seleção de um modelo e treinamento do mesmo.

## Classificador binário

Um classificador binário simplesmente dá uma resposta SIM ou NÃO para uma instância. Então nossos rótulos também tem que ter este formato.

Por exemplo, o classificador pode indicar se a imagem associada a uma instância representa ou não um **dígito cinco**.

In [ ]:
y_train_5 = (y_train == 5)
y_test_5  = (y_test  == 5)

Pode-se começar com o classificador por Descida de Gradiente Aleatório (Stochastic
Gradient Descent - SGD), usando a classe `SGDClassifier` da biblioteca Scikit-Learn.

Este classificador tem a vantagem de ser capaz de lidar eficientemente com conjuntos de dados grandes.
Isso ocorre em parte porque o SGD lida com instâncias de treinamento de forma independente, uma de cada vez
(o que também torna o SGD adequado para aprendizado _online_).

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
# predição para imagem de índice 1, que é um dígito 5
sgd_clf.predict([some_digit])[0]

## Medindo a acurácia usando validação cruzada

Avaliar um classificador é mais complicado do que avaliar um regressor, pois existem várias medidas de  desempenho que podem ser usadas.

A primeira medida a ser avaliada, usando **validação cruzada** do conjunto de treino, é a **acurácia** (_accuracy_).

In [ ]:
# medindo a acurácia usando validação cruzada, com 3 dobras
from sklearn.model_selection import cross_val_score

scores = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')
print(scores)

O resultado sempre acima de 93% de acurácia, que é o percentual de acertos parece muito bom!

Contudo é importante comparar este caso com um classificador trivial, que sempre responde **não** para a previsão de saída.

In [ ]:
# classificador trivial que responde negativamente a todas as instâncias
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
never_5_clf = Never5Classifier()
scores = cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring='accuracy')
print(scores)

Ou seja, mais de 90% de acurácia. Isso ocorre simplesmente porque apenas cerca de 10% das imagens são dígitos 5, então se o classificador semrpe supor que uma imagem não é um 5, ele estará certo 90% do tempo.

Isso mostra que a acurácia geralmente não é a medida de desempenho mais adequada para classificadores, especialmente quando se está lidando com conjuntos de dados não-uniformemente distribuídos (ou seja, quando algumas das classes são muito mais frequentes do que outras).

## Matriz de confusão

Assim como a função `cross_val_score()`, a função `cross_val_predict()` validação cruzada, mas em vez de retornar as pontuações da avaliação, esta última retorna as previsões feitas em cada dobra do teste.

Isso significa que você obtém uma previsão clara para cada instância no conjunto de treinamento, justamente pela  previsão ter sido feita por um modelo que nunca viu tal instância durante o treinamento.

In [ ]:
# predição para todas as instâncias, usando validação cruzada
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

Cada linha em uma **matriz de confusão** representa uma classe real, enquanto cada coluna representa uma classe prevista.

In [ ]:
# geração da matriz de confusão
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_5, y_train_pred)
print(cm)

In [ ]:
print('verdadeiros negativos (VN):', cm[0, 0])
print('falsos positivos      (FP):', cm[0, 1])
print('falsos negativos      (FN):', cm[1, 0])
print('verdadeiros positivos (VP):', cm[1, 1])

Um classificador perfeito teria apenas verdadeiros positivos e verdadeiros negativos. Então sua matriz de confusão teria valores diferentes de zero apenas em sua diagonal:

In [ ]:
# classificação perfeita: igual aos valores reais
y_train_perfect_predictions = y_train_5
print(confusion_matrix(y_train_5, y_train_perfect_predictions))

Uma métrica interessante é a **precisão** (_precision_), definida assim:
    
\begin{equation}
\text{precisão} = \frac{VP}{VP+FP}
\end{equation}

Outra métrica chamada de **sensibilidade** (_sensitivity_) ou **revocação** (_recall_), é a proporção de instâncias positivas que são detectadas corretamente pelo classificador:

\begin{equation}
\text{sensibilidade} = \frac{VP}{VP+FN}
\end{equation}

In [ ]:
# cálculo da precisão e da sensibilidade
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

In [ ]:
recall_score(y_train_5, y_train_pred)

A leitura que pode ser feita é que quando o classificador afirma que uma imagem representa um dígito 5, ele está **correto apenas 72,6% das vezes**. Mais ainda, este classificador detecta **apenas 81,1% dos dígitos 5 presentes**.

Muitas vezes, é conveniente combinar precisão e sensibilidade em uma única métrica chamada **pontuação F1** (_F1 score_), em paerticular para poder comparar dois classificadores.


A pontuação F1 é a média harmônica da precisão e da sensibilidade. Enquanto a média regular trata todos os valores igualmente, a média harmônica dá muito mais peso aos valores baixos:

\begin{equation}
F_1 = \frac{VP}{VP+\frac{FN+FP}{2}}
\end{equation}

Como resultado, o classificador só obterá uma pontuação F1 alta se ambos, recall e sensibilidade forem altos.

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)

Note que a pontuação F1 favorece classificadores que têm precisão e sensibilidade semelhantes.

Isso nem sempre é o que se deseja: em alguns contextos, é importante a precisão e, em outros, a sensibilidade.

## Outro classificador binário

Agora será usado o modelo `RandomForestClassifier`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)

In [ ]:
# predição para todas as instâncias, usando validação cruzada
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(forest_clf, X_train, y_train_5, cv=3)

In [ ]:
# geração da matriz de confusão
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_5, y_train_pred)
print('verdadeiros negativos (VN):', cm[0, 0])
print('falsos positivos      (FP):', cm[0, 1])
print('falsos negativos      (FN):', cm[1, 0])
print('verdadeiros positivos (VP):', cm[1, 1])

In [ ]:
# medidas de desempenho
from sklearn.metrics import precision_score, recall_score, f1_score

print('precisão:     ', precision_score(y_train_5, y_train_pred))
print('sensibilidade:', recall_score(y_train_5, y_train_pred))
print('pontuação f1: ', f1_score(y_train_5, y_train_pred))

# Classificação multiclasse


É possível construir um classificador multiclasse usando vários classificadores binários.

Felizmente a biblioteca Scikit-Learn escolhe a estratégia automaticamente, dependendo do algoritmo selecionado.

Basta então criar o classificados e treiná-no com os rótulos completos em `y_train`, ao invés dos rótulos binários `y_train_5` usados anteriormente.

O efeito prático é que classificadores multiclasse levarão mais tempo para serem treinados. Quanto mais classes envolvidas, mais demorado será.

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)

# esta chamada pode demorar um pouco
%time sgd_clf.fit(X_train, y_train)

In [ ]:
# predição para imagem de índice 1, que é um dígito 5
sgd_clf.predict([some_digit])[0]

Para entender como funciona um classificador multiclasse:

1. Foram treinados 10 classificadores (um para cada classe).
2. A entrada então foi aplicada em cada um dos 10, gerando 10 pontuações.
3. A previsão é dada pelo rótulo associado ao classificador com a melhor pontuação.

In [ ]:
# pontuações individuais dos 10 classificadores internos
some_digit_scores = sgd_clf.decision_function([some_digit])
print(some_digit_scores)

In [ ]:
# qual o índice que contém o maior score?
index = np.argmax(some_digit_scores)

In [ ]:
# classe prevista
sgd_clf.classes_[index]

## Medida de desempenho por validação cruzada

In [ ]:
# esta chamada pode demorar um pouco
%time cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring='accuracy')

## Análise do erro

Aqui podemos gerar a previsão para todas as instâncias, usando validação cruzada. Então podemos comparar o real com o previsto em uma matriz de confusão para todas as 10 classes.

In [ ]:
# esta chamada pode demorar um pouco
%time y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)
cm = confusion_matrix(y_train, y_train_pred)
cm

In [ ]:
# exibição da matriz de confusão como imagem
plt.matshow(cm, cmap='gray')
plt.show()

Os resultados em geral são bons, pois a todos os dígitos foram em geral identificados como os próprios, como indica os valores mais claros na diagonal.

Para visualizar o erro nos demais casos, a matriz será normalizada e os elementos da diagonal serão zerados.

In [ ]:
# normalizar a matriz
row_sums = cm.sum(axis=1, keepdims=True)
norm_cm = cm / row_sums

# zerar diagonal e exibir
np.fill_diagonal(norm_cm, 0)
plt.matshow(norm_cm, cmap='gray')
plt.show()

As linhas representam as classes reais, enquanto as colunas representam as classes previstas.

A coluna da classe 8 é bem clara, o que indica que muitas imagens foram classificadas incorretamente como dígitos 8. No entanto, a linha para a classe 8 não é tão ruim, informando que em geral os dígitos 8 reais são classificados corretamente como tais.

Note que a matriz de confusão não é necessariamente simétrica. Também pode ser observado que os dígitos 3 e 8 muitas vezes são erroneamente classificados como 5.

In [ ]:
# exbição de algumas instâncias de 5 e 8
cl_a, cl_b = 5, 8
X_aa = X_train.values[(y_train == cl_a) & (y_train_pred == cl_a)]
X_ab = X_train.values[(y_train == cl_a) & (y_train_pred == cl_b)]
X_ba = X_train.values[(y_train == cl_b) & (y_train_pred == cl_a)]
X_bb = X_train.values[(y_train == cl_b) & (y_train_pred == cl_b)]

plt.figure(figsize=(8,8))
plt.subplot(221); plot_digits(X_aa[:25], images_per_row=5)
plt.subplot(222); plot_digits(X_ab[:25], images_per_row=5)
plt.subplot(223); plot_digits(X_ba[:25], images_per_row=5)
plt.subplot(224); plot_digits(X_bb[:25], images_per_row=5)
plt.show()

# Avaliar o modelo final

Basta usar o modelo anteriormente treinado, chamado `sgd_clf` e fazer uma previsão usando os dados de teste `X_test`, gerando as previsões  em `y_test_pred`.

As medidas de desempenho são obtidas comparando `y_test_pred` e `y_test` (que são os rótulos corretos de teste).

Vamos aqui usar as medidas de precisão, sensibilidade e pontuação F1, só que agora ajustadas para um classificador multiclasse (usando o parâmetro `average='weighted'`, que pondera as medidas de todas as classes).

In [ ]:
# predição para todo o conjunto de teste
y_test_pred = sgd_clf.predict(X_test)

# medidas de desempenho
from sklearn.metrics import precision_score, recall_score, f1_score
print('precisão:     ', precision_score(y_test, y_test_pred, average='weighted'))
print('sensibilidade:', recall_score(y_test, y_test_pred, average='weighted'))
print('pontuação f1: ', f1_score(y_test, y_test_pred, average='weighted'))